In [ ]:
import sys
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

from random import choices
from scipy import stats

sys.path.insert(0, '../src/')
from geospatial import read_gis

In [ ]:
import arcgis

# Arrests

In [ ]:
map_df = gpd.read_file('a6f2e9.shp')
map_df.head()

In [ ]:
arrests = pd.read_csv('../data/cleaned/arrests-processed.csv')
arrests.head()

In [ ]:
map_df.APREC.unique()

In [ ]:
arrests['Area Name'].unique()

In [ ]:
def format_div(x):
    if x.upper() in ['WEST LA','N HOLLYWOOD']:
        if x.upper()=='WEST LA':
            return 'WEST LOS ANGELES'
        else:
            return 'NORTH HOLLYWOOD'
    else:
        return x.upper()
arrests['Area Name'] = arrests['Area Name'].apply(format_div)

In [ ]:
arrests = arrests.loc[arrests.Year!=2020].groupby(['Area Name','Year']).size().unstack().reset_index()

In [ ]:
merged = map_df.set_index('APREC').join(arrests.set_index('Area Name'))
merged.head()

In [ ]:
years = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
output_path = ''
# set the min and max range for the choropleth map
vmin, vmax = 3000, 9000

for year in years:
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = merged.plot(column=year, cmap='Blues', figsize=(10,10), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax, 
                      legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    
    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title('Annual Arrests in LAPD Divisions', \
              fontdict={'fontsize': '25',
                         'fontweight' : '3'})
    
    # position the annotation to the bottom left
    fig.annotate(year,
            xy=(0.1, .225), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=35)
    
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, str(year)+'_arrests.jpg')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300)

# Interactive Map

In [ ]:
user = arcgis.GIS(username='cyyeh_ucsd5', password='Serpents123!')
item = user.content.get('031d488e158144d0b3aecaa9c888b7b3')
sdf = pd.DataFrame.spatial.from_layer(item.layers[0])
sdf.APREC.unique()

In [ ]:
def clean_csv(x):
    if x == 'West LA':
        return 'WEST LOS ANGELES'
    elif x == 'N Hollywood':
        return 'NORTH HOLLYWOOD'
    return x.upper()

In [ ]:
arrest_types = pd.read_csv('../viz/Analysis/Arrests/div_Charge Group Description_dist.csv')
arrest_types.iloc[:,0] = arrest_types.iloc[:,0].apply(clean_csv)
arrest_types.set_index('Unnamed: 0', inplace=True)
arrest_types.head()

In [ ]:
merged = sdf.set_index('APREC').join(arrest_types)
#merged.drop(columns=['OBJECTID','PREC','AREA','PERIMETER','Shape__Area','Shape__Length']).reset_index().spatial.to_featurelayer('PredPol Arrests Analysis', gis=user)

In [ ]:
import json
from bokeh.io import output_file, show, output_notebook
from bokeh.models import GeoJSONDataSource, LinearColorMapper, HoverTool, Select, CustomJS
from bokeh.plotting import figure
from bokeh.palettes import Viridis6
from bokeh.layouts import column

In [ ]:
output_notebook()

In [ ]:
with open(('crimes_analysis_json.geojson')) as fp:
    crime_types = json.load(fp)
    
geo_source = GeoJSONDataSource(geojson=json.dumps(crime_types))
color_mapper = LinearColorMapper(palette=Viridis6)
TOOLTIPS = [('Division', '@APREC'),('Impact', '@Personal')]
op2 = [('Division', '@APREC'),('Impact', '@Property')]
op3 = [('Division', '@APREC'),('Impact', '@Inchoate')]
op4 = [('Division', '@APREC'),('Impact', '@Statutory')]
op5 = [('Division', '@APREC'),('Impact', '@Financial_')]
option = [TOOLTIPS, op2,op3,op4,op5]

output_file("crime_arrest_types.html")
a = figure(title='Change in Proportion of Crime Types per Division', tooltips=TOOLTIPS, x_axis_location=None, y_axis_location=None, width=500, height=500)
b = a.patches('xs', 'ys', fill_alpha=0.7, fill_color={'field': 'Personal', 'transform': color_mapper},
          line_width=0.5, source=geo_source)

with open(('arrests_analysis.geojson')) as fp:
    arrest_types = json.load(fp)
    
geo_source2 = GeoJSONDataSource(geojson=json.dumps(arrest_types))
c = figure(title='Change in Proportion of Arrest Types per Division', tooltips=TOOLTIPS, x_axis_location=None, y_axis_location=None, width=500, height=500)
d = c.patches('xs', 'ys', fill_alpha=0.7, fill_color={'field': 'Personal', 'transform': color_mapper},
          line_width=0.5, source=geo_source2)

callback = CustomJS(args=dict(tt=a.hover, tt2=c.hover, opts=option, renderer=b, renderer2=d, cmap=color_mapper), code="""
        if (cb_obj.value=='Personal') {
            tt[0].tooltips=opts[0];
            tt2[0].tooltips=opts[0];
        } else if (cb_obj.value=='Property') {
            tt[0].tooltips=opts[1];
            tt2[0].tooltips=opts[1];
        } else if (cb_obj.value=='Inchoate') {
            tt[0].tooltips=opts[2];
            tt2[0].tooltips=opts[2];
        } else if (cb_obj.value=='Statutory') {
            tt[0].tooltips=opts[3];
            tt2[0].tooltips=opts[3];
        } else {
            tt[0].tooltips=opts[4];
            tt2[0].tooltips=opts[4];
        }
        
        if (cb_obj.value!='Financial/Other') {
            renderer.glyph.fill_color = {'field': cb_obj.value, 'transform': cmap}
            renderer2.glyph.fill_color = {'field': cb_obj.value, 'transform': cmap}
        } else {
            renderer.glyph.fill_color['field'] = {'field': 'Financial_', 'transform': cmap}
            renderer2.glyph.fill_color['field'] = {'field': 'Financial_', 'transform': cmap}
        }
        renderer.trigger('change')
    """)

select = Select(title="Crime Type:", value="Personal", options=["Personal", "Property", "Inchoate", "Statutory", "Financial/Other"])
select.js_on_change('value', callback)

layout = column(select, a, c)
show(layout)